In [1]:
from gurobipy import *
import pandas as pd
import numpy as np
import requests
from io import StringIO
m = Model('mip1')

Academic license - for non-commercial use only


### Data Wrangling

In [2]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/dermatology/dermatology.data'
names = ('erythema', 'scaling', 'definite borders', 'itching', 'koebner phenomenon','polygonal papules',
        'follicular papules', 'oral mucosal involvement', 'knee and elbow involvement', 'scalp involvement',
        'family history', 'melanin incontinence', 'eosinophils in the infiltrate', 'PNL infiltrate',
        'fibrosis of the papillary dermis', 'exocytosis', 'acanthosis', 'hyperkeratosis', 'parakeratosis',
        'clubbing of the rete ridges', 'elongation of the rete ridges', 'thinning of the suprapapillary epidermis',
        'spongiform pustule', 'munro microabcess', 'focal hypergranulosis', 'disappearance of the granular layer',
        'vacuolisation and damage of basal layer', 'spongiosis', 'saw-tooth appearance of retes', 'follicular horn plug',
        'perifollicular parakeratosis', 'inflammatory monoluclear inflitrate', 'band-like infiltrate', 'Age', 'class')
remote = requests.get(url).content
data_df = pd.read_csv(StringIO(remote.decode('utf-8')), names = names)


In [19]:
data_df.head()

,erythema,scaling,definite borders,itching,koebner phenomenon,polygonal papules,follicular papules,oral mucosal involvement,knee and elbow involvement,scalp involvement,...,disappearance of the granular layer,vacuolisation and damage of basal layer,spongiosis,saw-tooth appearance of retes,follicular horn plug,perifollicular parakeratosis,inflammatory monoluclear inflitrate,band-like infiltrate,Age,class
0,2,2,0,3,0,0,0,0,1,0,...,0,0,3,0,0,0,1,0,55,2
1,3,3,3,2,1,0,0,0,1,1,...,0,0,0,0,0,0,1,0,8,1
2,2,1,2,3,1,3,0,3,0,0,...,0,2,3,2,0,0,2,3,26,3
3,2,2,2,0,0,0,0,0,3,2,...,3,0,0,0,0,0,3,0,40,1
4,2,3,2,2,2,2,0,2,0,0,...,2,3,2,3,0,0,2,3,45,3


In [4]:
def nullify(x):
    if x == '?':
        return np.nan
    else:
        return x

In [5]:
data_df = data_df.applymap(nullify)
data_df = data_df.dropna(axis = 0,how = 'any').reset_index(drop = True)

In [16]:
classification_X = data_df.drop(columns = ['class'])
classification_Y = data_df['class'] - 1

classification_X = classification_X.astype(str)
classification_X = pd.get_dummies(classification_X)
classification_X.head()

,erythema_0,erythema_1,erythema_2,erythema_3,scaling_0,scaling_1,scaling_2,scaling_3,definite borders_0,definite borders_1,...,Age_63,Age_64,Age_65,Age_67,Age_68,Age_7,Age_70,Age_75,Age_8,Age_9
0,0,0,1,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
from sklearn.model_selection import train_test_split
X_train_df, X_test_df, Y_train_df, Y_test_df = train_test_split(classification_X, classification_Y, test_size = 0.20)

In [8]:
X_train_df = X_train_df.reset_index(drop = True)
Y_train_df = Y_train_df.reset_index(drop = True)
X_test_df = X_test_df.reset_index(drop = True)
Y_test_df = Y_test_df.reset_index(drop = True)

X_train = X_train_df.values
X_test = X_test_df.values
Y_train = Y_train_df.values
Y_test = Y_test_df.values

rows, cols = X_train.shape

### Determine the Depth

In [9]:
depth = 2

### CART Tree

In [10]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(max_depth = depth, max_leaf_nodes = 2**depth)
clf.fit(X_train_df, Y_train_df)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=4, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [11]:
total_nodes = clf.tree_.node_count
leaf_nodes = round(total_nodes / 2)
branch_nodes = total_nodes // 2

In [12]:
left_nodes = clf.tree_.children_left
right_nodes = clf.tree_.children_right
left_nodes = left_nodes[0:branch_nodes]
right_nodes = right_nodes[0:branch_nodes]

In [13]:
feature = clf.tree_.feature
initial_a = []
initial_a_tmp = []
for i in range(len(feature)):
    if feature[i] != -2 and feature[i+1] != -2:
        initial_a.append(feature[i])
    elif feature[i] != -2 and feature[i+1] == -2:
        initial_a_tmp.append(feature[i])
    else:
        continue
initial_a.extend(initial_a_tmp)
initial_a

[73, 109, 53]

In [14]:
threshold = clf.tree_.threshold
initial_b = []
initial_b_tmp = []
for i in range(len(threshold)):
    if threshold[i] != -2 and threshold[i+1] != -2:
        initial_b.append(threshold[i])
    elif threshold[i] != -2 and threshold[i+1] == -2:
        initial_b_tmp.append(threshold[i])
    else:
        continue
initial_b.extend(initial_b_tmp)
initial_b

[0.5, 0.5, 0.5]

In [15]:
clf.score(X_test_df, Y_test_df)

0.7777777777777778

### OCT

### Determine the alpha and K

In [16]:
alpha = 0.5
K = 6

In [17]:
max_diff = np.max(X_train, 0) - np.min(X_train, 0)
epsilon = np.ones([1, cols], dtype = int) * max_diff
for i in range(cols):
    old = 0
    for j in range(1, rows):
        diff = abs(X_train[j, i] - X_train[old ,i])
        old = j
        if diff < epsilon[0, i] and diff != 0:
            epsilon[0, i] = diff

epsilon = abs(epsilon)
epsilon

array([[0.33333333, 0.33333333, 0.33333333, 0.33333333, 0.33333333,
        0.33333333, 0.33333333, 0.33333333, 0.33333333, 0.33333333,
        1.        , 0.33333333, 0.5       , 0.33333333, 0.33333333,
        0.33333333, 0.33333333, 0.33333333, 0.33333333, 0.33333333,
        0.33333333, 0.33333333, 0.33333333, 0.33333333, 0.33333333,
        0.33333333, 0.33333333, 0.33333333, 0.33333333, 0.33333333,
        0.33333333, 0.33333333, 0.33333333, 0.01333333]])

In [18]:
max_epsilon = np.max(epsilon)

n = rows
num_feature = cols

### Determine the Y matrix

In [19]:
Y = np.zeros([rows,K], dtype = int) - 1

Y[X_train_df.index, Y_train_df.astype(int)] = 1

Y

array([[-1,  1, -1, -1, -1, -1],
       [ 1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1,  1, -1],
       ...,
       [-1,  1, -1, -1, -1, -1],
       [-1, -1, -1,  1, -1, -1],
       [ 1, -1, -1, -1, -1, -1]])

In [20]:
l = m.addVars(leaf_nodes, vtype = GRB.BINARY, name = "l")

z = m.addVars(n, leaf_nodes, vtype = GRB.BINARY, name = "z")

N_kt = m.addVars(K, leaf_nodes, vtype = GRB.INTEGER, name = "N_kt")

N_t = m.addVars(leaf_nodes, vtype = GRB.INTEGER, name = "N_t")

c_kt = m.addVars(K, leaf_nodes, vtype = GRB.BINARY, name = "c")

L = m.addVars(leaf_nodes, vtype = GRB.INTEGER, name = "L")

a = m.addVars(branch_nodes, num_feature, vtype = GRB.BINARY, name = 'a')

b = m.addVars(branch_nodes ,vtype = GRB.CONTINUOUS, name = "b")

d = m.addVars(branch_nodes ,vtype = GRB.BINARY, name = "d")

### Warm Start

In [21]:
# warm start using the results of CART algorithm

for i in range(branch_nodes):
    a[i, initial_a[i]].start = 1
    b[i].start = initial_b[i]

In [22]:
m.update()

In [23]:
m.setObjective(L.sum() + alpha * d.sum(), GRB.MINIMIZE)

### OCT Constraints

In [24]:
for i in range(branch_nodes):
    b[i].setAttr(GRB.Attr.LB, 0)
    m.addConstr(b[i] <= d[i])
    m.addConstr(a.sum(i, '*') == d[i])
    m.addConstr(d[i] == 1)
    m.addConstr(l[i] == 1)
    
for i in range(leaf_nodes):
    m.addConstr(L[i] >= 0)
    m.addConstr(N_t[i] == z.sum('*', i))
    m.addConstr(l[i] == c_kt.sum('*', i))
    m.addConstr(z.sum('*', i) >= l[i])
    for j in range(K):
        m.addConstr(L[i] >= N_t[i] - N_kt[j,i] - n * (1 - c_kt[j,i]))
        m.addConstr(L[i] <= N_t[i] - N_kt[j,i] + n * c_kt[j,i])
        m.addConstr(N_kt[j,i] == 1/2 * sum(z.select('*', i) * (Y[:,j] + 1)))

for i in range(n):
    m.addConstr(z.sum(i, '*') == 1)
    
for i in range (leaf_nodes):
    for j in range(n):
        m.addConstr(z[j,i] <= l[i])

all_branch_nodes = list(reversed(range(branch_nodes)))
depth_dict = {}
for i in range(depth):
    depth_dict[i] = sorted(all_branch_nodes[-2**i:])
    for j in range(2**i):
        all_branch_nodes.pop()

all_leaf_nodes = list(range(leaf_nodes))
branch_dict = {}
for i in range(branch_nodes):
    for k in range(depth):
        if i in depth_dict[k]:
            floor_len = len(depth_dict[k])
            step = 2**depth // floor_len
            sliced_leaf = [all_leaf_nodes[i:i+step] for i in range(0, 2**depth, step)]
            idx = depth_dict[k].index(i)
            branch_dict[i] = sliced_leaf[idx]
        else:
            continue
            
for j in range(n):
    for i in range(leaf_nodes):
        for k in range(branch_nodes):
            if i in branch_dict[k]:
                length = len(branch_dict[k])
                idx = branch_dict[k].index(i)
                if idx+1 <= length//2:
                    m.addConstr(sum(a.select(k, '*') * (X_train[j,:] + epsilon[0,:])) <= b[k] + (1 + max_epsilon) * (1 - z[j,i]))
                elif idx+1 > length//2:
                    m.addConstr(sum(a.select(k, '*') * X_train[j,:]) >= b[k] - (1 - z[j,i]))
            else:
                continue

In [ ]:
m.optimize()

Optimize a model with 3818 rows, 1312 columns and 66489 nonzeros
Variable types: 3 continuous, 1309 integer (1277 binary)
Coefficient statistics:
  Matrix range     [1e-02, 3e+02]
  Objective range  [5e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Presolved: 2944 rows, 1306 columns, 62545 nonzeros

Continuing optimization...



In [ ]:
for v in m.getVars():
    print(v.varName, v.x)

In [ ]:
print('Obj:', m.objVal)

### Obtain the Tree Structure

In [ ]:
# Obtain the coefficients of a and b
coff_a = np.zeros([branch_nodes, num_feature], dtype = int)
# coff_b = np.zeros(branch_nodes, dtype = int)
coff_b = np.zeros(branch_nodes)

for i in range(branch_nodes):
    tmp1 = m.getVarByName('b' + '[' + str(i) + ']')
#     coff_b[i] = int(tmp1.x)
    coff_b[i] = tmp1.x
    for j in range(num_feature):
        tmp2 = m.getVarByName('a' + '[' + str(i) + ',' + str (j) + ']')
        coff_a[i,j] = int(tmp2.x)



In [29]:
# Obtain the labels of leaf nodes
labels = np.zeros(leaf_nodes, dtype = int) - 1
coff_c = np.zeros([K, leaf_nodes], dtype = int)

for i in range(K):
    for j in range(leaf_nodes):
        tmp3 = m.getVarByName('c' + '[' + str(i) + ',' + str (j) + ']')
        coff_c[i,j] = int(tmp3.x)

k_idx, t_idx = np.where(coff_c == 1)
# for i in range(leaf_nodes):
for i in range(len(k_idx)):
    labels[t_idx[i]] = k_idx[i]

### Test Data

In [30]:
t_rows, t_cols = X_test.shape

In [31]:
tmp = np.zeros([t_rows, 1], dtype = int)
Y_predict = np.hstack((np.reshape(Y_test_df.values, (t_rows, 1)), tmp))

In [32]:
num_nodes = 0
for i in range(branch_nodes): 
    tmp4 = m.getVarByName('d' + '[' + str(i) + ']')
    num_nodes += int(tmp4.x)

In [33]:
init = np.array([], dtype = int).reshape(0, num_feature)
nodes = {}
for i in range(num_nodes * 2):
    nodes[i] = init

# split
for i in range(t_rows):
    if np.dot(coff_a[0,:], np.transpose(X_test[i,:])) < coff_b[0]:
        nodes[0] = np.vstack([X_test[i,:], nodes[0]])
        if np.dot(coff_a[1,:], np.transpose(X_test[i,:])) < coff_b[1]:
            nodes[2] = np.vstack([X_test[i,:], nodes[2]])
            Y_predict[i,1] = labels[0]
            
        elif np.dot(coff_a[1,:], np.transpose(X_test[i,:])) >= coff_b[1]:
            nodes[3] = np.vstack([X_test[i,:], nodes[3]])
            Y_predict[i,1] = labels[1]
            
    elif np.dot(coff_a[0,:], np.transpose(X_test[i,:])) >= coff_b[0]:
        nodes[1] = np.vstack([X_test[i,:], nodes[1]])
        if np.dot(coff_a[2,:], np.transpose(X_test[i,:])) < coff_b[2]:
            nodes[4] = np.vstack([X_test[i,:], nodes[4]])
            Y_predict[i,1] = labels[2]
            
        elif np.dot(coff_a[2,:], np.transpose(X_test[i,:])) >= coff_b[2]:
            nodes[5] = np.vstack([X_test[i,:], nodes[5]])
            Y_predict[i,1] = labels[3]



### Test Accuracy

In [34]:
1 - sum(np.minimum(abs(Y_predict[:,1] - Y_predict[:,0]), 1)) / t_rows

0.9